In [ ]:
import numpy as np
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD


#Dataset
train_path1='../input/emotiondetectiondataset/Expression_dataset/FER dataset/train'
test_path1='../input/emotiondetectiondataset/Expression_dataset/FER dataset/test'

train_path2='../input/emotiondetectiondataset/Expression_dataset/CK+48/train'
test_path2='../input/emotiondetectiondataset/Expression_dataset/CK+48/test'

combined_train ='../input/emotiondetectiondataset/Expression_dataset/combined/train'
combined_test = '../input/emotiondetectiondataset/Expression_dataset/combined/test'

equiTest='../input/equitestfinal/TestSet'

global_test = '../input/emotion/FinalTestSet'

# def myFunc(image):
#     image = np.array(image)
#     image= np.expand_dims(image, axis=0)
#     image = image.astype('float32')
#     image /= 255
#     return image
#Splitting dataset
#train_datagen = ImageDataGenerator(preprocessing_function=keras.applications.inception_resnet_v2.preprocess_input, validation_split=0.2) 

from keras.models import load_model
#Change model
model = keras.applications.mobilenet.MobileNet(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)
x = model.layers[-2].output
predictions = Dense(6,activation='softmax')(x)
localmodel1 = Model(inputs=model.input,outputs=predictions)

localmodel1.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
#model.summary()
#Change 2 lines
preproce=keras.applications.mobilenet.preprocess_input
print("dataset1")
train_batches1= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(train_path1,target_size=(224,224),batch_size=32,shuffle=True)
test_batches1= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(test_path1,target_size=(224,224),batch_size=32,shuffle=False)
print("dataset2")
train_batches2= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(train_path2,target_size=(224,224),batch_size=32,shuffle=True)
test_batches2= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(test_path2,target_size=(224,224),batch_size=32,shuffle=False)
print("combined")
combined_train= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(combined_train,target_size=(224,224),batch_size=32,shuffle=True)
combined_test= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(combined_test,target_size=(224,224),batch_size=32,shuffle=False)
print("globalTest")
global_test= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(global_test,target_size=(224,224),batch_size=32,shuffle=False)
equi_Test= ImageDataGenerator(preprocessing_function=preproce).flow_from_directory(equiTest,target_size=(224,224),batch_size=32,shuffle=False)

In [ ]:
x = model.layers[-2].output
predictions = Dense(6,activation='softmax')(x)
localmodel2 = Model(inputs=model.input,outputs=predictions)
localmodel2.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
x = model.layers[-2].output
predictions = Dense(6,activation='softmax')(x)
globalModel = Model(inputs=model.input,outputs=predictions)
globalModel.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import tensorflow as tf
def scale_model_weights(weight):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(0.5 * weight[i])
    return weight_final
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

In [ ]:
val_loss_client1=[]
val_acc_client1=[]
val_loss_client2=[]
val_acc_client2=[]
val_loss_global1=[]
val_acc_global1=[]
val_loss_global2=[]
val_acc_global2=[]
val_loss_globalTest=[]
val_acc_globalTest=[]

for i in range (25):
    global_weights = globalModel.get_weights()
    scaled_local_weight_list = list()
    localmodel1.set_weights(global_weights)
    localmodel2.set_weights(global_weights)
    his = localmodel1.fit(train_batches1,steps_per_epoch=np.ceil(train_batches1.samples / train_batches1.batch_size),epochs =1 ,verbose=1,validation_data = test_batches1,validation_steps=np.ceil(test_batches1.samples / test_batches1.batch_size))
    loss, acc = localmodel1.evaluate_generator(test_batches1, steps=np.ceil(test_batches1.samples / test_batches1.batch_size), verbose=0)
    print('client 1 accuracy at ', i ,":",acc)
    val_loss_client1.append(loss)
    val_acc_client1.append(acc)
    his = localmodel2.fit(train_batches2,steps_per_epoch=np.ceil(train_batches2.samples / train_batches2.batch_size),epochs =1 ,verbose=1,validation_data = test_batches2,validation_steps=np.ceil(test_batches2.samples / test_batches2.batch_size))
    loss, acc = localmodel2.evaluate_generator(test_batches2, steps=np.ceil(test_batches2.samples / test_batches2.batch_size), verbose=0)
    print('client 2 accuracy at ', i ,":",acc)
    val_loss_client2.append(loss)
    val_acc_client2.append(acc)
    scaled_weights = scale_model_weights(localmodel1.get_weights())
    scaled_local_weight_list.append(scaled_weights)
    scaled_weights = scale_model_weights(localmodel2.get_weights())
    scaled_local_weight_list.append(scaled_weights)
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    #update global model 
    globalModel.set_weights(average_weights)
    loss, acc = globalModel.evaluate_generator(test_batches1, steps=np.ceil(test_batches1.samples / test_batches1.batch_size), verbose=1)
    print('Global Model accuracy at 1 ', i ,":",acc)
    val_loss_global1.append(loss)
    val_acc_global1.append(acc)
    loss, acc = globalModel.evaluate_generator(test_batches2, steps=np.ceil(test_batches2.samples / test_batches2.batch_size), verbose=1)
    print('Global Model accuracy at 2 ', i ,":",acc)
    val_loss_global2.append(loss)
    val_acc_global2.append(acc)
    loss, acc = globalModel.evaluate_generator(global_test, steps=np.ceil(global_test.samples / global_test.batch_size), verbose=1)
    print('Global Model accuracy at globatestset ', i ,":",acc)
    val_loss_globalTest.append(loss)
    val_acc_globalTest.append(acc)
    avg_acc = (val_acc_global1[i] + val_acc_global2[i] + val_acc_globalTest[i])/3
    print("Average:",avg_acc)
    globalModel.save("%d-%f.h5" % (i, val_acc_globalTest[i]))
    print("----------------------------------------------------------")
    
df = pd.DataFrame(list(zip(val_loss_client1, val_acc_client1,val_loss_client2,val_acc_client2,val_loss_global1,val_acc_global1,val_loss_global2,val_acc_global2,val_loss_globalTest,val_acc_globalTest)),
               columns =['Loss_client1', 'Acc_client1','Loss_client2','Acc_client2','val_loss_global1','val_acc_global1','val_loss_global2','val_acc_global2','val_loss_globalTest','val_acc_globalTest'])
df.to_csv("loss_acc.csv")
    
    
    

In [ ]:
df = pd.DataFrame(list(zip(val_loss_client1, val_acc_client1,val_loss_client2,val_acc_client2,val_loss_global1,val_acc_global1,val_loss_global2,val_acc_global2,val_loss_globalTest,val_acc_globalTest)),
               columns =['Loss_client1', 'Acc_client1','Loss_client2','Acc_client2','val_loss_global1','val_acc_global1','val_loss_global2','val_acc_global2','val_loss_globalTest','val_acc_globalTest'])
df.to_csv("loss_acc.csv")

In [ ]:
df.head()

In [ ]:
x = model.layers[-2].output
predictions = Dense(3,activation='softmax')(x)
new = Model(inputs=model.input,outputs=predictions)

new.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
localmodel1.save("merged.h5")

In [ ]:
his = localmodel1.fit(combined_train,steps_per_epoch=np.ceil(combined_train.samples / combined_train.batch_size),epochs =1 ,verbose=1,validation_data = combined_test,validation_steps=np.ceil(combined_test.samples / combined_test.batch_size))


In [ ]:
loss, acc = new.evaluate_generator(test_batches1, steps=np.ceil(test_batches1.samples / test_batches1.batch_size), verbose=0)

In [ ]:
acc

In [ ]:
#generate training curve
import matplotlib.pyplot as plt
import pandas as pd
history = his
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
df = pd.DataFrame(list(zip(train_acc, val_acc,train_loss,val_loss)),
               columns =['Training Acc', 'Validation Acc','Training Loss','Validation Loss'])
df.to_csv('curve.csv')
epochs = range(1,26)
plt.plot(epochs, train_acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
#plt.title('Attention LSTM-CNN (FastText Embedding)')
plt.xlabel('Epochs', fontsize='medium')
plt.ylabel('Accuracy', fontsize='medium')
plt.legend()
#sn.set(font_scale=1)
plt.savefig('train.pdf', format='pdf', dpi=300)
plt.savefig('train.png', format='png', dpi=300)
plt.show()

In [ ]:
#generate training curve
#import matplotlib.pyplot as plt
#history = his
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1,51)
plt.plot(epochs, train_acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
#plt.title('Attention LSTM-CNN (FastText Embedding)')
plt.xlabel('Epochs', fontsize='medium')
plt.ylabel('Accuracy', fontsize='medium')
plt.legend()
#sn.set(font_scale=1)
plt.savefig('train.pdf', format='pdf', dpi=300)
plt.savefig('train.png', format='png', dpi=300)
plt.show()

In [ ]:
local

In [ ]:
#generate Result
from __future__ import print_function
import sklearn
from matplotlib import pyplot as plt 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

#Change HERE
#model = load_model('./17-0.678674.h5')
model=localmodel1
#combined_test
#equi_Test
test_batches=equi_Test
loss, acc = model.evaluate_generator(test_batches, steps=np.ceil(test_batches.samples / test_batches.batch_size), verbose=1)
print('accuracy:',acc)
predictions = model.predict_generator(test_batches, steps = np.ceil(test_batches.samples / test_batches.batch_size), verbose=1, workers=0) 
Y_pred = np.argmax(predictions, axis=1) 
print('Classification Report') 
print(classification_report(test_batches.classes, Y_pred))
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
original = test_batches.labels
cm=confusion_matrix(original,Y_pred)
print(cm)
#y_true = ["0","1","2"]
y_true=['Anger', 'Disgust','Fear', 'Happy','Sad','Surprise']
data = cm
# class1_acc = data[0][0]/(data[0][0]+data[0][1]+data[0][2])
# class2_acc = data[1][1]/(data[1][0]+data[1][1]+data[1][2])
# class3_acc = data[2][2]/(data[2][0]+data[2][1]+data[2][2])
# print('class-1 acc: ',class1_acc)
# print('class-2 acc: ',class2_acc)
# print('class-3 acc: ',class3_acc)
for i in range(0,6):
    acc = data[i][i]
    s =0
    for j in range(0,6):
        s = s+data[i][j]
    acc = acc/s
    print("Class %d accuracy - %f" % (i, acc))


df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
#df_cm.index.name = 'Actual'
#df_cm.columns.name = 'Predicted'
sn.set(font_scale=1)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 18}, fmt="d")
plt.savefig('Cm.pdf', format='pdf', dpi=300)
plt.savefig('Cm.png', format='png', dpi=300)


